In [2]:
from PyPDF2 import PdfFileReader
import os
import json

Extract information from pdf file

In [2]:
def extract_pdf(path):
    pdf=PdfFileReader(path)
    number_pages=pdf.getNumPages()
    content=''
    for i in range(number_pages):
        content+=pdf.getPage(i).extractText()
    
    content=content.replace('\\n',' ').replace('\n\n',' ')
    content=content.replace('\n',' ')
    content=content.replace('\t',' ')
    
    return content

Get path to each pdf file stored in folder  

In [3]:

def get_path_pdf(path_folder):
    path_pdf=[]
    for dirname,_,filenames in os.walk(path_folder):
        for filename in filenames:
            path_pdf.append(os.path.join(dirname,filename))
    return path_pdf


Read JD

In [4]:
path_JD="./JD/"
path_2_pdfJD=get_path_pdf(path_JD)
content_JD=extract_pdf(path_2_pdfJD[0])


Read CV

In [5]:
path_CV="./CV/"
path_2_pdfCV=get_path_pdf(path_CV)
content_CV=[]
for p in path_2_pdfCV:
    content_CV.append(extract_pdf(p))


Using Spacy to extract skill

In [6]:
from spacy.pipeline import EntityRuler
from spacy import displacy
import jsonlines
import spacy
import re 
import pyperclip

In [7]:
json_path="skill_paterns.jsonl"

with jsonlines.open(json_path) as f:
    entities=[line["label"].upper() for line in f.iter()]

# spacy model
nlp=spacy.load("en_core_web_lg")

In [8]:
ruler = nlp.add_pipe("entity_ruler")
ruler.from_disk(json_path)
nlp.pipe_names

['tok2vec',
 'tagger',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'ner',
 'entity_ruler']

In [9]:
doc=nlp(content_CV[0])

def get_skill(doc):
    skill_list=[]
    for ent in doc.ents:
        if ent.label_=='SKILL':
            skill_list.append(ent.text)
    skill_list=list(set(skill_list))
    return skill_list
    
get_skill(doc) 

['Computer vision',
 'computer science',
 'Python',
 'OpenGL',
 'CERTIFICATE',
 'ML',
 'Google',
 'OpenCV',
 'database',
 'Deep Learning',
 'languages',
 'SQL',
 'data structure',
 'Computer Science',
 'algorithm',
 'Numpy',
 'C',
 'C++',
 'design']

Read Personal infomation

Experience

In [49]:
from sympy import true


doc=nlp(content_CV[0])

def get_GPA(doc):
    sent=list(doc.sents)
    gpa=''
    for s in sent:
        s=s.text
        if s.find('GPA') !=-1 :
            gba_pos= s.find('GPA') 
            for i in range (gba_pos+3,gba_pos+8):
                if (s[i] >'0' and s[i]<'9' or s[i]=='.'):
                    gpa+=s[i]
    return gpa

def get_experience(doc):
    '''
    Return\n
        None if no experience\n
        List of experience if exist
    '''
    sent=list(doc.sents)
    final_res=''
    ex=[]
    for s in sent:
        s_text=s.text.split(' ')
        flag=False
        for i in s_text:
            i=i.lower()
            if i=='experience':
                flag=True
        if flag==True:
            ex.append([s.text])
    if (len(ex)==0):
        final_res=None
    else:
        final_res=ex
    return final_res

def get_education(doc):
    sent=list(doc.sents)
    edu=''
    pos_edu=-1
    for s in sent:
        s_text=s.text.lower()
        flag=False
        if s_text.find('education')!=-1 or s_text.find('university')!=-1:
            edu =s.text
            if s_text.find('education')!=-1:
                pos_edu=s_text.find('education')
            else:
                pos_edu=s_text.find('university')
            
            edu=edu[pos_edu+10:]
    return edu

print(get_GPA(doc))       
print(get_experience(doc))
print()
print(get_education(doc))
     

7.5
None



'Bachelor of Computer Science (Current GPA: 7.5/10) 2019-2023 University of Science Learn the computer science foundation knowledge such as programing languages, database, data structure and algorithm, math, Computer vision, Machine learning, AI, Recognition SKILLS Framework/Libraries OpenCV, Numpy, Pytorch, OpenGL Language C/C++, Python, C#, SQL Soft Skills Teamwork, Presentation, English (Upper Intermediate) PROJECTS March 2022MNIST'

PERSONAL INFORMATION

In [11]:
test = content_CV[0]
mail = re.compile(r'[a-zA-Z0-9-\.]+@[a-zA-Z-\.]*\.(com|edu|net)')
git = re.compile(r'(gitlab.com/|github.com/)+[a-zA-Z0-9-\.]*')
web = re.compile(r'(http://|https:// )+[a-zA-Z0-9-"/"-\.]*')
linkedin = re.compile(r'linkedin.com/+[a-zA-Z0-9-"/"-\.]*')

def get_phone():
    '''
    Format for Phone-Numbers:
    134567890
    123-456-7890
    (123)(456)(7890)
    123.456.7890
    (123)(456-7890)
    '''
    phone= re.compile(r'''
        (\d{3}|\(\d{3}\))    
        (\s|-|\.)?           
        (\d{3}|\(\d{3}\))    
        (\s|-|\.)?           
        (\d{4}|\(\d{4}\))    
    ''', re.VERBOSE) 
    
    matchs=phone.finditer(test)
    res = ''
    for match in matchs:
        res+=match.group(0)
    return res

def get_link(link):
    matchs=link.finditer(test)
    res=''
    for match in matchs:
        res+=match.group(0)
    return res

print(get_link(web))
print(get_link(git))
print(get_link(linkedin))
print(get_link(mail))
get_phone()



linkedin.com/in/bich-ngoc-0ab1831b3
Ptbichngoc2502@gmail.com


'0347297090'

NAME

In [12]:
from spacy.matcher import Matcher
# initialize matcher with a vocab
matcher = Matcher(nlp.vocab)
def extract_name(content_CV):
    doc=nlp(content_CV)
    # First name and Last name are always Proper Nouns
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
    matcher.add('NAME',[pattern])
    matches = matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]
        return span.text

extract_name(content_CV[0])

'Phan Thi'

In [13]:
nlp_model=spacy.load('nlp_model')

In [24]:
def get_name(doc):
    result=''
    for ent in doc.ents:
        if ent.label_=='PERSON':
            result=f'{ent.label_.upper():{30}}-{ent.text}'
        break

    return result


Check Accuracy of name

In [27]:
train_data = json.load(open('./data/training/train_data.json','rb')) 
list_an=[]
list_txt=[]
name=[]
for text,annotation in train_data:
    list_an.append(list(annotation.values()))
    list_txt.append(text)

name=[]
for i in range(len(list_an)):
    for j in range(len(list_an[i][0])):
        if list_an[i][0][j][2]=='Name':
            name.append(list_txt[i][list_an[i][0][j][0]:list_an[i][0][j][1]])

name

['Govardhana K',
 'Harini Komaravelli',
 'Hartej Kathuria',
 'Ijas Nizamuddin',
 'Imgeeyaul Ansari',
 'Jay Madhavi',
 'Jitendra Babu',
 'Jyotirbindu Patnaik',
 'Karthihayini C',
 'Karthik GV',
 'Kartik Sharma',
 'Kasturika Borah',
 'Kavitha K',
 'Kavya U.',
 'Khushboo Choudhary',
 'kimaya sonawane',
 'Koushik Katta',
 'Kowsick Somasundaram',
 'Lakshika Neelakshi',
 'Madas Peddaiah',
 'Madhuri Sripathi',
 'Mahesh Vijay',
 'Manisha Bharti',
 'Manjari Singh',
 'Mohamed Ameen',
 'Mohini Gupta',
 'Navas Koya',
 'Navjyot Singh Rathore',
 'Nazish Alam',
 'Nidhi Pandit',
 'Nikhileshkumar Ikhar',
 'Nitin Tr',
 'Pradeeba V',
 'Prakriti Shaurya',
 'PRASHANTH BADALA',
 'Pratibha P',
 'Prem Koshti',
 'Pulkit Saxena',
 'Puneet Singh',
 'Rahul Bollu',
 'Rajeev Kumar',
 'Ram Edupuganti',
 'Ramesh HP',
 'Ramya. P',
 'R Arunravi',
 'Ravi Shankar',
 'Ravi Shivgond',
 'Rohit Bijlani',
 'Roshan Sinha',
 'Sai Dhir',
 'Sai Patha',
 'Sai Vivek Venkatraman',
 'Sameer Kujur',
 'Samyuktha Shivakumar',
 'Santosh 

Using sklearn to calculate the cosine similarity score of CV and JD

In [50]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cv=CountVectorizer()
test=[content_CV[2],content_JD]
count_matrix=cv.fit_transform(test)
cosine_similarity(count_matrix)

In [ ]:
count_matrix

<2x314 sparse matrix of type '<class 'numpy.int64'>'
	with 340 stored elements in Compressed Sparse Row format>